# Import Libraries

In [3]:
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input, Lambda, Flatten, Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [4]:
# resize image
IMAGE_SIZE = [224, 224]

In [5]:
# define data path
train_path = "Datasets/Train"
test_path = "Datasets/Test"

In [7]:
# add preprocessing layer to front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 59s 1us/step


In [8]:
# don't train existing layers
for layer in vgg.layers:
    layer.trainable = False